# Faster RCNN Training Notebook

## Import Libraries

In [1]:
import os, sys
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.append(BASE_DIR)

from src.models.Detection.Faster_RCNN import Faster_RCNN
from src.dataset.bdd_detetcion import BDDDetection
from src.config.defaults import cfg
from pytorch_lightning import Trainer
from pytorch_lightning.profiler import SimpleProfiler
from pytorch_lightning.utilities.model_summary import ModelSummary
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from src.utils.DataLoaders import get_loader

## Load Datasets and DataLoaders

In [ ]:
obj_cls = ['__bgr__', 'pedestrian', 'motorcycle', 'car', 'bicycle']

In [2]:
bdd_train_params = {
    'cfg': cfg,
    'stage': 'train',
    'obj_cls': obj_cls
}

bdd_train = BDDDetection(**bdd_train_params)

100%|██████████████████████████████████████████████████████████████████████████| 69863/69863 [00:09<00:00, 7029.43it/s]


In [3]:
bdd_val_params = {
    'cfg': cfg,
    'stage': 'val',
    'obj_cls': obj_cls
}

bdd_val = BDDDetection(**bdd_val_params)

100%|██████████████████████████████████████████████████████████████████████████| 69863/69863 [00:09<00:00, 7345.31it/s]


In [4]:
train_dataloader_args = {
    'dataset': bdd_train,
    'batch_size': 32,
    'shuffle': True,
    'collate_fn': bdd_train.collate_fn
}
train_dataloader = get_loader(**train_dataloader_args)

val_dataloader_args = {
    'dataset': bdd_val,
    'batch_size': 32,
    'shuffle': False,
    'collate_fn': bdd_train.collate_fn
}
val_dataloader = get_loader(**val_dataloader_args)

## Load Faster RCNN Model

In [15]:
faster_rcnn_params = {
    'cfg': cfg,
    'num_classes': len(bdd_train.cls_to_idx),
    'backbone': 'resnet101',
    'learning_rate': 1e-5,
    'weight_decay': 1e-3,
    'pretrained': True,
    'pretrained_backbone': True,
    'checkpoint_path': None,
    'train_loader': train_dataloader,
    'val_loader': val_dataloader
}
model = Faster_RCNN(**faster_rcnn_params)

C:\Users\Khaliladib\anaconda3\envs\yoloc\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'backbone_name' and 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\Khaliladib\anaconda3\envs\yoloc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Model Summary

In [ ]:
ModelSummary(model, max_depth=-1)  

## Training Section

In [ ]:
profiler = SimpleProfiler()
early_stop_callback = EarlyStopping(monitor="val_loss", patience=5, verbose=False, mode="min")
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode='min')

In [ ]:
#trainer = Trainer(auto_lr_find=True, limit_train_batches=0.0001, limit_val_batches=0.01, profiler=profiler)

In [ ]:
trainer = Trainer(auto_lr_find=True, profiler=profiler, callbacks=[early_stop_callback, checkpoint_callback])

In [ ]:
trainer.tune(model)

In [ ]:
trainer.fit(model)